In [3]:
from pathlib import Path

In [4]:
BASE="/tmp/foobar"

In [16]:
extension = "pth"
best_name = "best_model"
best_tpl = "best_model_"
ckpt_tpl = "checkpoint_"

In [30]:
repo_id = "jimregan/ckpt_test"

In [8]:
BASE_PATH = Path(BASE)

In [1]:
from huggingface_hub import HfApi

/Users/joregan/opt/anaconda3/envs/hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
api = HfApi()

In [18]:
def get_ckpt_num(ckpt_path: Path, best=False):
    if best:
        tpl = best_tpl
    else:
        tpl = ckpt_tpl
    stem = ckpt_path.stem
    if not stem.startswith(tpl):
        return None
    return int(stem.replace(tpl, ""))

In [9]:
ckpts_by_numbers = {get_ckpt_num(x): x for x in BASE_PATH.glob(f'{ckpt_tpl}*.{extension}')}

In [12]:
ckpts_ordered = sorted(ckpts_by_numbers.keys())

In [19]:
best_ordered = sorted([get_ckpt_num(x, True) for x in BASE_PATH.glob(f'{best_tpl}*.{extension}')])

In [20]:
best_ordered

[9800, 28852, 64434, 64867, 65300]

In [23]:
def find_next_ckpt(best):
    for ckpt in ckpts_ordered:
        if ckpt < best:
            continue
        else:
            return ckpt

ckpt_to_best = {find_next_ckpt(b): b for b in best_ordered}

In [24]:
ckpt_to_best

{10000: 9800, 29000: 28852, 64500: 64434, 65000: 64867, 65500: 65300}

In [27]:
from huggingface_hub import CommitOperationAdd

In [ ]:
ops = []
for filepath in BASE_PATH.glob("*"):
    if not filepath.stem.startswith(ckpt_tpl) and not filepath.stem.startswith("best_tpl"):
        ops.append(CommitOperationAdd(path_in_repo=filepath.name, path_or_fileobj=filepath))

api.create_commit(
    repo_id=repo_id,
    operations=ops,
    commit_message="Initial commit of files other than checkpoints"
)

In [ ]:
for ckpt in ckpts_ordered:
    if ckpt in ckpt_to_best:
        print("Best", ckpt_to_best[ckpt])
    print("Ckpt", ckpt)

```
create_branch

( repo_id: str, branch: str, revision: Optional[str] = None, token: Optional[str] = None, repo_type: Optional[str] = None, exist_ok: bool = False )
```